In [1]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, top)
# sys.path

In [2]:
# -*- coding: utf-8 -*-
import asyncio
import requests
import aiohttp
from pyquery import PyQuery as pq
import pandas as pd
import os
import sys
from datetime import datetime
import itertools
import random
from time import time
#
from fastapi import Request
# 為了在jupyter中試，從apps開始import
import apps.ips.config as ips_cfg
from apps.ips.config import (
    url_free_cycle, level_https, cacert,
    ips_csv_path, ips_html_path,
    dtype, dt_format,
    ipcols, get_freeproxy_delta,
)
from apps.ips.utils import aio_get, write_file, csv_update, CHECK_PROXY
###############################################################################
###############################################################################


async def get_freeproxy(t, once=True):
    get_freeproxy_cnt = 0
    while 1:
        T = (ips_cfg.ips_cycle and os.path.isfile(ips_csv_path))*t  # 沒有 csv 或 ips_cycle 就馬上爬
        await asyncio.sleep(T)
        stime = time()
        #
        async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=cacert)) as session:
            status_code, rtext = await aio_get(session, next(url_free_cycle))
            if status_code == 200 and rtext not in ['', None]:
                doc = pq(rtext, parser='html')
                trs = doc.find('table.table').eq(0).find('tr')
                if trs.size():
                    now = datetime.today().strftime(dt_format)
                    elite = []
                    for tr in trs:
                        tds = pq(tr).find('td')
                        level = tds.eq(4).text().strip()
                        https = tds.eq(6).text().strip()
                        # _________________________________
                        if (level, https) not in level_https:
                            continue
                        # _________________________________
                        tmp = {
                            'ip': tds.eq(0).text().strip(),
                            'port': tds.eq(1).text().strip(),
                            'now': now,
                            'goodcnt': 0,
                        }
                        elite.append(tmp)
                    # 1 儲存每次撈取的原始頁面 #################################
                    write_file(ips_html_path, rtext)
                    # 2 重存csv: 讀取csv檔案，與最新爬的比較 #################################
                    if os.path.isfile(ips_csv_path):
                        df1 = pd.read_csv(ips_csv_path, dtype=dtype)
                        df2 = pd.DataFrame(elite).astype(dtype)
                        df3 = csv_update(df1, df2)
                    else:
                        df3 = pd.DataFrame(elite).astype(dtype)
                    # 3 檢查代理 #################################
                    ippts = df3[ipcols].values.tolist()  # goodcnt不傳進CHECK_PROXY
                    print(f'\n開始檢查proxy:共 {len(ippts)} 個')
                    good_proxys = await CHECK_PROXY.get_good_proxys(ippts)
                    print(f'結束檢查proxy: {time()-stime}')
                    random.shuffle(good_proxys)
                    # 4 存 csv #################################
                    df3 = pd.DataFrame(good_proxys).astype(dtype)  # df.sample(frac=1)  # 亂排
                    df3.to_csv(ips_csv_path, index=False)
                    # 5 更新ips_cycle產生器 #################################
                    ips_cfg.ips_cycle = itertools.cycle(good_proxys)
                    #
                    get_freeproxy_cnt += 1
                    print(f'get_freeproxy 第{get_freeproxy_cnt}次更新成功:{now}')
                    print(f'good_proxys 數量: {len(good_proxys)}\n')
                else:
                    pass
        if once:
            break


###############################################################################
tasks_list = [
    (get_freeproxy, [get_freeproxy_delta, False]),
]
###############################################################################


if __name__ == '__main__':
    try:
        tmp = 'zmqshell' in str(type(get_ipython()))  # 在jupyter
        print(tmp, '在jupyter')
    except Exception:
        loop = asyncio.get_event_loop()
        task = loop.create_task(get_freeproxy(1))
        loop.run_until_complete(task)
else:
    if 0:
        asyncio.create_task(get_freeproxy(3, False))

True 在jupyter


In [3]:
await get_freeproxy(1, True)


開始檢查proxy:共 115 個
結束檢查proxy: 22.169406175613403
get_freeproxy 第1次更新成功:2021-03-31_09:34:49
good_proxys 數量: 55



In [4]:
from apps.book.classes.bbooks import BOOKS

{'__module__': 'apps.book.classes.bbooks', '__qualname__': 'BOOKS', 'info_default': {'bookid': '0010770978'}, 'url_target_prefix': 'https://www.books.com.tw/products/', '__init__': <function BOOKS.__init__ at 0x7f304a7f5040>, 'update_info': <function BOOKS.update_info at 0x7f304a7f5b80>, 'save_info': <function BOOKS.save_info at 0x7f304a7f5310>, 'author_handle': <function BOOKS.author_handle at 0x7f304a7f51f0>, '__classcell__': <cell at 0x7f307418bb50: empty>}


In [5]:
init = {'bookid': '0010556358'}
book = BOOKS(**init)

In [6]:
await book.update_info()

{'store': 'BOOKS', 'bookid': '0010556358', 'isbn10': None, 'isbn13': '9789573270140', 'title': '大亨小傳', 'title2': 'The Great Gatsby', 'author': '作者：費茲傑羅/原文作者：F. Scott Fitzgerald/譯者：汪芃', 'publisher': None, 'pub_dt': None, 'lang': None, 'price_list': None, 'price_saleprice_list_ebook': None, 'price_sale_ebook': None, 'spec': None, 'intro': None, 'url_book': None, 'url_vdo': None, 'url_cover': None, 'err': None, 'create_dt': '2021-03-31_09:35:15'}
3.683809757232666


In [10]:
15//2

7

In [34]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
url = 'https://www.books.com.tw/products/0010770978?sloc=main'
#
connector = aiohttp.TCPConnector(ssl=True)
TO = aiohttp.ClientTimeout(total=11)
async with aiohttp.ClientSession(connector=connector, timeout=TO) as session:
    async with session.get(url, headers=headers) as r:
        status = r.status
        rtext = await r.text(encoding='utf8')

In [35]:
doc = pq(rtext, parser='html')
doc

[<html>]

In [40]:
doc.find('.cont iframe').eq(0).attr('src') is None

True

In [30]:
rtext

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">\n<html>\n<head>\n<meta http-equiv="content-type" content="text/html; charset=utf-8">\n<meta http-equiv="Content-Language" content="zh-tw">\n<title>博客來-21世紀的21堂課</title>\n<meta name="keywords" content="21世紀的21堂課">\n<meta name="description" content="書名：21世紀的21堂課，原文名稱：21 Lessons for the 21st Century，語言：繁體中文，ISBN：9789864795192，頁數：400，出版社：天下文化，作者：哈拉瑞，譯者：林俊宏，出版日期：2018/08/30，類別：社會科學">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta property="og:site_name" content="博客來"/>\n<meta property="og:type" content="product"/>\n<meta property="fb:app_id" content="210743125726405" />\n<meta property="og:title" content="21世紀的21堂課"/>\n<meta property="og:image" content="https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/081/57/0010815762.jpg&v=5c7666fe&w=1146&h=600"/>\n<meta property="og:image:width" content="1146">\n<meta property="og:image:height" content="600">\n<meta property=